In [1]:
# 2018-05-16
# A. Pendleton
# Parsing the BLATs of the non-predicted RefSeq (release #87) mammalian genes against the zoey2.3 genome

In [2]:
#this uses iPython magic to make plots appear inline
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import subprocess
import sys
import numpy as np
import matplotlib.patches as patches
import gzip
import fileinput
import glob
from scipy import stats
import re
from matplotlib_venn import venn3, venn3_circles
from collections import OrderedDict


def count_lines(f):
    lineCount = 0
    with open(f, 'r') as f:
        for line in f:
            lineCount += 1
        return lineCount
def runCMD(cmd):
    val = subprocess.Popen(cmd, shell=True).wait()
    if val == 0:
        pass
    else:
        print ('command failed')
        print (cmd)
        sys.exit(1)
# TO REMOVE TOP AND RIGHT AXIS OF PLOTS
def simpleaxis(ax):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()

# First read in the list of IDs to link the RefSeq ID to the gene name

In [8]:
refSeqDir = '/nfs/turbo/jmkidddata/ampend-projects/annotations_genes/refseq-mammalian-release87/'
refSeqFile = refSeqDir + 'NonPredicted_refseq87_TotalIDs.txt'

refDict = {}
count = 0

for line in open(refSeqFile,'r'):
    count +=1 
    line=line.replace('>','')
    #print(line.rstrip())
    refID = line.split(' ')[0] #Refseq ID
    fullDesc = line.split(' ')[1]
    #Get gene name (alias)
    #geneName = re.findall('\((.+)\),', line)[-1]
    #print(line)
    if '(' in line:
        g = re.search("\((\S*)\),", line)
        G = str(g.group(0))
        geneName = G.replace('(','').replace(')','').replace(',','')
        #print(G,geneName)
    else:
        geneName = ''
    #Get the gene description
    Detail = line.split(' ')[-1]

    #TEST = d.lastgroup()
    if refID in refDict.keys():
        print('Error: redundant refseq ID = ', refID)
        break
    else:
        refDict[refID] = [line, geneName, Detail]

    #if count > 100:
    #    break
print('%i genes processed ' % count)

171160 genes processed 


In [ ]:
IDFile = open(refSeqDir + 'RefSeqID_to_GeneName_conversion.txt','w')

for refID in refDict.keys():
    print(refID)
    print(refDict[refID])
    IDFile.write()
    break
IDFile.close()

# Parsing in the BLAT results

#Methods per UCSC:



The RNAs were aligned against the dog genome using blat; those with an alignment of less than 15% were discarded. When a single RNA aligned in multiple places, the alignment having the highest base identity was identified. Only alignments having a base identity level within 0.5% of the best and at least 25% base identity with the genomic sequence were kept.



In [44]:
blatDir = '/home/ampend/links/kidd-lab/ampend-projects/Zoey_Genome_Project/annotations/BLAT_with_RefSeqMammalian/results/'

blatDict = {}
allHits = []
seqCount = 0 

for file in glob.glob(blatDir + '*.psl'):
    #print(file)
    for line in open(file,'r'):
        seqCount += 1
        
        line=line.rstrip().split()
        score,orient,gene,geneLength = int(line[0]),line[8],line[9],int(line[10])
        chrom,start,end = line[13],int(line[15]),int(line[16]) 
        
        blockCount = int(line[17])
        blocks = line[18].split(',')
        
        blockSum = 0
        for i in range(0,len(blocks)-1):
            j = int(blocks[i])
            blockSum = blockSum + j
        
        propHit = float(blockSum / geneLength)
        #skip all "hits" with less than 15% of length alignment (from adding up blocks)
        if propHit < 0.15:
            continue

        #First hit automatically gets saved since it has a higher propHit than required (15%)
        if gene not in blatDict.keys():
            blatDict[gene] = []
            blatDict[gene] = [chrom,start,end,gene,orient,geneLength,blockSum,propHit,blockCount,line[18],line[19]]
            continue
        
        #amount different between this hit and top hit
        bestHit = blatDict[gene][7]
        propDifference = abs(propHit - bestHit)
        
        #if the hit is better than the best hit
        if propDifference < 0.5 and propHit > 0.25:
            allHits.append([chrom,start,end,gene,orient,geneLength,blockSum,propHit,blockCount,line[18],line[19]])
        
    
    for key in blatDict.keys():
        info = allHits.append(blatDict[key])
    #break

print('%i top hits in dictionary' % len(blatDict.keys()))
print('%i hits (top + close hits) in array' % len(allHits))

57292 top hits in dictionary
4802932 hits (top + close hits) in array
